In [ ]:
%matplotlib inline
import scipy.special, cmath
import random, math, pylab, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pylab
import mpl_toolkits.mplot3d

# Trotter approximation: application to the real time evolution of the system

In order to study the real time evolution of a quantum system, we need to revert to the description using wave functions, and the Schrodinger equation $i \dfrac{\partial}{\partial t} \psi(t) = H  \psi(t)$, where $H$ is the Hamiltonian of the system. This equation can be formally solved as $\psi(t) = \exp(-i t H) \psi_0$. In this form, $exp(-i t H)$ appears as a time-evolution operator. Moreover, the substitution $it \to \beta$, i.e., the use of "imaginary time", transforms this time-evolution operator into the density matrix operator. This formal similarity allows us to draw a parallel between the results established in equilibrium as a function of inverse temperature, and the time evolution of the system. In particular, the results related to the Trotter expansion can be expanded

This time, we can decompose the full time interval $[0, \ldots, t]$ into steps of size $\Delta t = t / N$, to obtain $$ \exp(-itH) = \prod\limits_{k=1}^{N}\exp(-i \Delta t H) = \prod\limits_{k=1}^{N}\exp(-i \Delta t (H^{\text{free}} + V(x))$$

Now, application of the Trotter decomposition, with $\beta \to i\Delta t$, leads to $$\exp(-i \Delta t (H^{\text{free}} + V(x)) = \exp(-i \frac{\Delta t}{2} V)  \exp(-i \Delta t H^{\text{free}}) \exp(-i \frac{\Delta t}{2} V) + \mathcal{O}(\Delta t^3)$$

After handling the middle term in momentum space, we find that $\exp(-i \Delta t H^{\text{free}}) \hat{\psi}(p, t) = \exp(-i \Delta t \frac{p^2}{2}) \hat{\psi}(p, t)$. With this, we are able to devise an algorithm for the determination of the time-evolution of the system.

Provided: The grids in real and momentum space.

In [ ]:
# Total number of grid points in real space and in momentum space.
x_steps = 800
# max and min values of x
x_min = -12.0
x_max = 40.0
# definition of the grid for discretization of the wave function in x coordinates
grid_x = np.linspace(x_min, x_max, x_steps)
dx  = grid_x[1] - grid_x[0]
# definition of the grid for discretization of the wave function in momentum space
grid_p = np.linspace(x_min, x_max, x_steps)
dp  = grid_p[1] - grid_p[0]

Provided: Fourier transforms, backwards and forwards.

In [ ]:
def fourier_x_to_p(phi_x, dx):
    phi_p = [(phi_x * np.exp(-1j * p * grid_x)).sum() * dx for p in grid_p]
    return np.array(phi_p)

def fourier_p_to_x(phi_p, dp):
    phi_x = [(phi_p * np.exp(1j * x * grid_p)).sum() * dp for x in grid_x]
    return np.array(phi_x) /  (2.0 * np.pi)

Provided: Specific shape of the potential: Two barriers
* One "inifinite" barrier at $x = -8$, 
* One small barrier at $x = 0$

In [ ]:
def funct_potential(x):
    if x < -8.0:    return (x + 8.0) ** 2
    elif x <= -1.0: return 0.0
    elif x < 1.0:   return np.exp(-1.0 / (1.0 - x ** 2)) / np.exp(-1.0)
    else:           return 0.0

Provided: graphical representation of the solution.
* x is the grid provided in the skeleton
* proba is the presence probability of the particle at $x$, i.e. the square of the modulus of the normalized wave function, in the form of numpy array.
* pot is the potential, static, in the same form as given in the skeleton below (numpy array)
* time is the current time value, a real number.
* time_step is the number of time steps we have executed to reach the current time, an integer.

In [ ]:
output_dir = 'snapshots_time_evolution'
if not os.path.exists(output_dir): os.makedirs(output_dir)
def show(x, proba, pot, time, timestep):
    # Plot the probability of presence at x
    pylab.plot(x, proba, 'g', linewidth = 2.0, label = '$|\psi(x)|^2$')
    pylab.xlim(-10, 15)
    pylab.ylim(-0.1, 1.15)
    # plot the potential
    pylab.plot(x, pot, 'k', linewidth = 2.0, label = '$V(x)$')
    pylab.xlabel('$x$', fontsize = 20)
    # show the current time in the title
    pylab.title('time = %s' % time)
    pylab.legend(loc=1)
    # Use the time step index to create the snapshot filename
    pylab.savefig(output_dir + '/snapshot_%05i.png' % timestep)
    timestep += 1
    pylab.show()
    pylab.clf()

# EXERCISE: 

## Implement the evolution of the wave function psi0 over the elementary time interval $\Delta_t$

In [ ]:
def time_step_evolution(psi0, potential, grid_x, grid_p, dx, dp, delta_t):
    # Here, implement the steps described in class
    return psi0

Provided: skeleton for the main loop.

In [ ]:
# definition of the time step
delta_t = 0.05
t_max = 16.0

# initialization of the potential barriers
potential = np.array([funct_potential(x) for x in grid_x])

# initial state of the wave function: some random Gaussian packet
x0 = - 8.0
sigma = .5
psi = np.exp(-(grid_x - x0) ** 2 / (2.0 * sigma ** 2) )
psi /= np.sqrt( sigma * np.sqrt( np.pi ) )

# time evolution
time = 0.0
timestep = 0
while time < t_max:
    if timestep % 5 == 0:
        show(grid_x, np.absolute(psi) ** 2.0, potential, time, timestep)
    time += delta_t
    timestep += 1
    psi = time_step_evolution(psi, potential, grid_x, grid_p, dx, dp, delta_t)
print time

# LEVY Construction

## Zoom on $x_k$: study of the distribution $\pi(x_k | x', x'')$

In [ ]:
def rho_free(x, y, beta):
    return math.exp(-(x - y) ** 2 / (2.0 * beta))

dtau_prime  = 0.1
dtau_dprime = 0.2
x_prime  = 0.0
x_dprime = 1.0
delta = 1.0                 # maximum displacement of xk
n_steps = 1000000            # number of Monte Carlo steps
data_hist = []
xk = 0.0                    # initial value of xk

# Metropolis algorithm
for step in xrange(n_steps):
    xk_new = xk + random.uniform(-delta, delta)
    old_weight  = (rho_free(x_dprime, xk, dtau_dprime) *
                   rho_free(xk, x_prime, dtau_prime))
    new_weight  = (rho_free(x_dprime, xk_new, dtau_dprime) * 
                   rho_free(xk_new, x_prime, dtau_prime))
    if random.random() < new_weight / old_weight:
        xk = xk_new
    data_hist.append(xk)

# Known analytic form
def pi_analytic(xk, x_prime, x_dprime, dtau_prime, dtau_dprime):
    mean = (dtau_dprime * x_prime + dtau_prime * x_dprime) / (dtau_prime + dtau_dprime)
    sigma = 1.0 / math.sqrt(1.0 / dtau_prime + 1.0 / dtau_dprime)
    return math.exp(-(xk - mean) ** 2 / (2.0 * sigma ** 2)) / math.sqrt(2.0 * math.pi) / sigma

pylab.title('Distribution on slice k', fontsize=18)
histo, bin_edges, dummy = pylab.hist(data_hist, bins=100, normed=True)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
pylab.plot(bin_centers, [pi_analytic(x, x_prime, x_dprime, dtau_prime, dtau_dprime) for x in bin_centers], 'r-', lw=3)
pylab.xlabel('$x_k$', fontsize=18)
pylab.ylabel('$\pi(x_k)$', fontsize=18)
pylab.savefig('plot-path_slice.pdf')

## Levy sampling for free particle

In [ ]:
#padding on the y axis between label and axis
plt.rcParams['ytick.major.pad']='10'

dpi_out = 400
fig_width = 3.39
golden_mean = (np.sqrt(5)-1.0)/2.0    # Aesthetic ratio
fig_height = fig_width*golden_mean # height in inches
MAX_HEIGHT_INCHES = 8.0
if fig_height > MAX_HEIGHT_INCHES:
    print("WARNING: fig_height too large:" + fig_height + 
          "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
    fig_height = MAX_HEIGHT_INCHES
#fig, ax = plt.subplots(figsize = (fig_width, fig_height), dpi=400, frameon=True)
small_tick_size = 8
small_label_size = 8

## EXERCISE: Implement the Levy construction for a free particle

In [ ]:
beta = 1.0
N = 50000                       # number of steps in imaginary time
dtau = beta / N                 # elementary time step in imaginary time
nsteps = 10                     # number of paths to be generated
xstart, xend = 0.0, 1.0         # initial and final points

for step in range(nsteps):
    # Sample a path

    fig, ax = plt.subplots(figsize = (fig_width, fig_height), dpi=400, frameon=True)
    ax.plot(x, np.linspace(0, 1, num=N + 1, endpoint=True))

## Continuous random walk

In [ ]:
beta = 4.0
N = 50000
sigma = math.sqrt(beta / N)
x = [0.0]
for k in range(N - 1):
    x.append(random.gauss(x[-1], sigma))

fig, ax = plt.subplots(figsize = (fig_width, fig_height), dpi=400, frameon=True)
ax.plot(x, np.linspace(0, 1, num=N, endpoint=True))

## Path sampling for a free particle from a continuous random walk

In [ ]:
beta = 1.0
N = 50000
sigma = math.sqrt(beta / N)
xend = 1.0
Upsilon = [0.0]
for k in range(N):
    Upsilon.append(random.gauss(Upsilon[-1], sigma))
x = [0.0] + [Upsilon[k] + (xend - Upsilon[-1]) * k / float(N) for k in range(1, N + 1)]

fig, ax = plt.subplots(figsize = (fig_width, fig_height), dpi=400, frameon=True)
ax.plot(x, np.linspace(0, 1, num=N+1, endpoint=True))